## Pytorch分布式API

对于进阶炼丹师，仅了解分布式训练是怎么启动的还不够。了解Pytorch分布式训练的一些API，能够帮助我们在自行编写一些需要分布式操作的代码时，更加游刃有余。

### All_reduce 操作

`all_reduce`是分布式通讯中最重要的一个概念之一。

1. 所谓的`reduce`，就是不同节点各有一份数据，把这些数据汇总到一起。在这里，我们规定各个节点上的这份数据有着相同的shape和data type，并规定汇总的方法是相加。简而言之，就是把各个节点上的一份相同规范的数据相加到一起。
2. 所谓的`all_reduce`，就是在reduce的基础上，把最终的结果发回到各个节点上。
3. 具体的`all_reduce`实现，要看具体的backend。流行的GPU backend NCCL，`all_reduce`的实现就是使用了ring思想。DDP利用all_reduce，来进行不同进程上的梯度的平均操作。

### DistributedSampler

分布式数据采样器是数据并行中的很重要一部分。不同进程所处理的数据集不能重叠，但每个epoch都会随机shuffle数据集来保证训练的性能。`DistributedSampler`利用随机数种子，达到两种shuffle的效果：

1. 横向来看，每个进程对于整个数据集（如ImageNet）的shuffle种子是固定的，即确保每个进程所处理的都是整个数据集互不重叠的子集；

2. 纵向来看，`DistributedSampler`把每个epoch的数值设置成采样种子，以这种方法来在训练迭代的过程中不断shuffle整个ImageNet，先整体按epoch数值shuffle，然后再针对每个进程固定shuffle。


#### RASamlpler

在Facebook发布的Deit论文中，他们对于分布式数据集采样提出了一种更新颖的方法：RASampler。

https://github.com/facebookresearch/deit
https://github.com/facebookresearch/deit/blob/main/samplers.py